# 1. Sarsa Algorithm and the OpenAI Gym's Taxi Environment

## Introduction

Tabular methods are suitable for small and discrete state space and discrete action space environments. So, the state-action function (Q) can be represented by a table of values. For large state space environments, we prefer to use approximation methods such as neural networks. However, the simplicity of tabular methods' implementation is helpful to demonstrate RL method's functionality. In this notebook, we train a SARSA agent for OpenAI's Taxi Gym environment .

## Goal

In this notebook, we are going to tackle the <a href="https://gym.openai.com/envs/Taxi-v2/">Taxi environment</a> (an example originally proposed by Tom Dietterich) with the Sarsa and Q-Learning algorithms. Once we implement Sarsa, implementing Q-Learning and many others would be easy. We usually need to change the core update method.

## The Taxi Environment

Here is a description of the taxi environment from the [docstring](https://github.com/openai/gym/blob/master/gym/envs/toy_text/taxi.py).

The Smartcab's job is to pick up the passenger in a simplified gridworld like environment at one location and drop them off in another ([Taxi, OpenAI Gym](https://gym.openai.com/envs/Taxi-v2/)). Here are a few things that we'd love our Smartcab to take care of:

- Drop off the passenger to the right location.
- Save passenger's time by taking minimum time possible to drop off
- Take care of passenger's safety and traffic rules

<center><img style="align: center;" src="https://github.com/FredAmouzgar/comp8220_ML_2021S1/raw/main/images/Taxi_Env.png" width=400></center>

__State__: Let's say we have a training area for our Smartcab where we are teaching it to transport people in a parking lot to four different locations (R, G, Y, B):

Let's assume Smartcab is the only vehicle in this parking lot. We can break up the parking lot into a 5x5 grid, which gives us 25 possible taxi locations. These 25 locations are one part of our state space. Notice the current location state of our taxi is coordinate (3, 1).

You'll also notice there are four (4) locations that we can pick up and drop off a passenger: R, G, Y, B or `[(0,0), (0,4), (4,0), (4,3)]` in (row, col) coordinates. Our illustrated passenger is in location Y and they wish to go to location R.

__Actions__: The agent is allowed to perform six possible actions:

1. south
2. north
3. east
4. west
5. pickup
6. dropoff

Notice in the illustration above, that the taxi cannot perform certain actions in certain states due to walls. In environment's code, we will simply provide a -1 penalty for every wall hit and the taxi won't move anywhere. This will just rack up penalties causing the taxi to consider going around the wall.

### Installations

In [ ]:
!pip -q install gym numpy matplotlib

### Setting up the environment

In [2]:
import gym

# Finding the Taxi environment
for env in gym.envs.registry.all():
    if env.id.startswith("Taxi"):
        env_name = env.id
##


print("Environment Name:", env_name)
env = gym.make(env_name)
env.reset()
env.render()

Environment Name: Taxi-v3
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



## The SARSA Algorithm
<p> </p><br>
<center><img style="align: center;" src="https://github.com/FredAmouzgar/comp8220_ML_2021S1/raw/main/images/SARSA_algorithm.png" width=900></center>

In [1]:
import numpy as np

class Sarsa_Agent:
    def __init__(self, states_n, actions_n, alpha=0.1, epsilon=0.1, gamma=0.95, epsilon_decay=True,
                 epsilon_decay_factor=0.01):
        self.alpha = alpha
        self.epsilon = epsilon
        self.gamma = gamma
        self.states_n = states_n
        self.actions_n = actions_n
        self.Q = np.zeros((states_n, actions_n))
        self.new_a = None
        self.epsilon_decay = epsilon_decay
        self.epsilon_decay_factor = epsilon_decay_factor

    def act(self, state):
        # epsilon greedy
        if np.random.rand() < self.epsilon:
            act = np.random.choice(np.arange(self.actions_n))
        else:
            act = np.argmax(self.Q[int(state), :])
        return act

    def decay_epsilon(self, factor):
        self.epsilon -= factor if self.epsilon >= 0 else 0

    def update(self, new_s, r, s, a, done):
        self.new_a = self.act(new_s)
        mask = 0 if done else 1
        s, a, self.new_a, new_s = int(s), int(a), int(self.new_a), int(new_s)
        self.Q[s, a] += self.alpha * (r + self.gamma * self.Q[new_s, self.new_a] * mask - self.Q[s, a])
        if done and self.epsilon_decay:
            self.decay_epsilon(self.epsilon_decay_factor)
        return self.new_a

## The Train Loop
<p> </p><br>
<center><img style="align: center;" src="https://github.com/FredAmouzgar/comp8220_ML_2021S1/raw/main/images/MDP_loop.jpeg" width=900></center>

# How the learning happened?

The Sarsa agent is implementing a __tabular method__ which means that it's updating a table of values iteratively. This table is called Q-table. After the training, the agent knows exactly which actions are the best in almost all possible states.

In __Approximation methods__ and __Deep Reinforcement Learning__, this table is replaced by a linear model or a [neural network](https://www.nature.com/articles/nature14236). A well-designed network can learn much more complex tasks, generalizes, and thus responds better to unseen situations.

__Take a look at the Pigoen Pong Experiment__ (special thanks go to <a href="https://complexity-in-action.github.io/people/patricknalepka.html">Dr. Patrick Nalepka</a>): Artificial agents follow the same process. __We'll show an agent with neural network on week 12.__

In [4]:
from IPython.display import HTML
HTML('<iframe width="560" height="315" src="https://www.youtube-nocookie.com/embed/vGazyH6fQQ4" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

<font size="1px">A modified version of our <a href="https://github.com/probml/pyprobml/blob/master/book2/supplements/rl/Tabular_SARSA.ipynb">joint work</a> with Prof. Kevin Murphy for the second edition of his book, <a href="https://probml.github.io/pml-book/">Machine Learning: A Probabilistic Perspective</a></font>

<font size="1px"><a href="https://www.linkedin.com/in/fredamouzgar/">Fred A.</a> Mar/2021</font>